In [1]:
from requests import request
from requests.compat import urljoin, urlparse
from requests.exceptions import HTTPError
from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
from time import sleep

In [2]:
def canfetch(url, agent='*', path='/'):
    robot = RobotFileParser(urljoin(url, '/robots.txt'))
    robot.read()
    return robot.can_fetch(agent, urlparse(url)[2])
    
def download(url, params={}, headers={}, method='GET', limit=3):
    if canfetch(url) == False:
        print('[Error] ' + url)
#     else: # 실제 수집할 때, 제약사항이 많으므로 여기선 잠시 해제
    try:
        resp = request(method, url,
               params=params if method=='GET' else {},
               data=params if method=='POST' else {},
               headers=headers)
        resp.raise_for_status()
    except HTTPError as e:
        if limit > 0 and e.response.status_code >= 500:
            print(limit)
            time.sleep(1) # => random
            resp = download(url, params, headers, method, limit-1)
        else:
            print('[{}] '.format(e.response.status_code) + url)
            print(e.response.status_code)
            print(e.response.reason)
            print(e.response.headers)
    return resp

### NAVER

In [3]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
params['query'] = '파 이 썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')

[Error] https://search.naver.com/search.naver


In [4]:
for _ in dom.find_all('ul', {'class' : 'type01'}):
    for a in [dt.find('a') for dt in _.find_all('dt')]:
        print(a['href'], a.text)

https://www.python.org/ Welcome to Python.org
https://blog.naver.com/icd900/221318575090 국산 중형차 추천 흰 파 썬
https://blog.naver.com/sanaiauction/221407173464 2015년형 그랜저HG 240모던 흰파썬 소개합니다!
https://blog.naver.com/xmflxhs8568?Redirect=Log&logNo=221532773930 레인보우 썬팅 전면,측.후면 12% 파.썬 30%
https://blog.naver.com/choyooney?Redirect=Log&logNo=221408338615 늦잠,티타임,GA과제,파에썬
https://blog.naver.com/sanaiauction?Redirect=Log&logNo=221407173464 2015년형 그랜저HG 240모던 흰파썬 소개합니다!
https://blog.naver.com/va85hgnmvcl?Redirect=Log&logNo=221687347280 [미얀마 만달레이 호텔] 썬 파 트래블 & 투어 인접한 더...
https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=357557482&qb=7YyMIOydtCDsjaw=&enc=utf8§ion=kin&rank=1&search_sort=0&spq=0  파이썬 if 문에서 and로 연결하는 방법 
https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=342332721&qb=7YyMIOydtCDsjaw=&enc=utf8§ion=kin&rank=2&search_sort=0&spq=0  파이썬 질문 ㅜㅜㅜ 
https://kin.naver.com/qna/detail.nhn?d1id=13&dirId=1303010501&docId=36135662&qb=7YyMIOydtCDsjaw=&enc=utf8§ion=kin&rank=3&search_

### Daum

In [5]:
url = 'https://search.daum.net/search'
params = {
    'w':'tot',
    'DA':'YZR',
    't__nil_searchbox':'btn',
    'sug':'',
    'sugo':'',
    'sq':'',
    'o':'',
    'q':''
}
params['q'] = '파 이 썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')

[Error] https://search.daum.net/search


In [6]:
# dom.find_all('ul', {'class' : 'list_info mg_cont clear'})
for _ in dom.find_all('div', {'class' : 'wrap_tit'}):
    print(_.find('a')['href'], _.find('a').text.strip())

https://keyword.ad.daum.net/short/clk?q=u.t3GAbqicNTvXihd4KIyTXGu7vitxYz5.WS8acgGKsEFFnUtKNKyPsCoxAVZ2.NSO4F7aYAODXggVTKzFDBS3jPMKK2UgM-yZWTA3vF_KTNoEOB8o4s5Y8jTd7onrJnPlb6n9hXSI3QS1BFd.O8Qh7TG-3rqRetVNFhgpLgodmv94IvMOAS6MLLsR_OZ-SGqT7zGfk.c_.E9mMLGijNNDpfeQ5oXNPvF2mIfzvrok3gzQ00&s=http%3A%2F%2Fwww.itbank21.com%2F%3FDMKW%3D%25ED%258C%258C%25EC%259D%25B4%25EC%258D%25AC%26DMSKW%3D%25ED%258C%258C%25EC%259D%25B4%25EC%258D%25AC%26DMCOL%3DPM KG아이티뱅크 교육문의 김서균
https://keyword.ad.daum.net/short/clk?q=u.-dRs7qWwxkc578ceq61giNHMIKtOc3rAKCDxnvpwxlZI5Jfj4hE4Cm8jCvoI3AKYdmPP.AgDkSGbOVYm.r-YmUJmXscvAQQ2O7.WsLUT9O5jdNJ4qB84Eic4Vm5st6_SVV6l7mmSDybJeJAC3O2nXt8kqXezCjoGwV3OypzEyER-vqGm5XGSsi1y-etcauiCPrDtFTU6NsgbZmq1cMfXq9ndK8iMmuN1v1cc5yEA3lrrUAbUGLHb-LhP3LefnSMQfGvl9Nt6Smx-EfeA9fKrnxDktuqmDYMlrx66kOYT.t1XVx9FH5YXX_lg3NYWIyjn_t5FyMu7-oXGT39WMo2K-qse1i88LzS3w0&s=https%3A%2F%2Fkmong.com%2Fcategory%2F605%3Futm_source%3Dpc_daum%26utm_medium%3Dcpc%26utm_campaign%3D6%26utm_content%3Dutmpage%26utm_term%3D%ED%8

### Crawling Practice

In [7]:
resp = download('http://pythonscraping.com/pages/page3.html')
dom = BeautifulSoup(resp.text, 'lxml')

In [8]:
dom.select_one('div#footer')

<div id="footer">
© Totally Normal Gifts, Inc. <br/>
+234 (617) 863-0736
</div>

In [9]:
dom.select_one('#footer')

<div id="footer">
© Totally Normal Gifts, Inc. <br/>
+234 (617) 863-0736
</div>

In [10]:
dom.select_one('#footer').text.strip()

'© Totally Normal Gifts, Inc. \n+234 (617) 863-0736'

- Selector
- tag1, tag2
- tag1 tag1 => 자손 (find_all(recursive=True))
- tag1 > tag2 => 자식 (file_all(recursive=False))
- tag1 + tag2 =>

In [11]:
dom.select_one('#footer').find_parent().name

'div'

In [12]:
len(dom.select('#wrapper > div')), len(dom.select('#wrapper > *'))

(2, 5)

In [13]:
[_.name for _ in dom.select('#wrapper > *')]

['img', 'h1', 'div', 'table', 'div']

In [14]:
dom.select_one('h1 + div').name

'div'

In [15]:
dom.select_one('h1 + div').find_previous_sibling().name

'h1'

In [16]:
dom.select_one('body > div > h1 + div').name

'div'

In [17]:
[_.text.strip() for _ in dom.select('.gift > td:nth-of-type(3)')]

['$15.00', '$10,000.52', '$10,005.00', '$0.50', '$1.50']

In [18]:
[_['src'].strip() for _ in dom.select('.gift > td > img')]

['../img/gifts/img1.jpg',
 '../img/gifts/img2.jpg',
 '../img/gifts/img3.jpg',
 '../img/gifts/img4.jpg',
 '../img/gifts/img6.jpg']

In [19]:
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
for _ in dom.find_all('h3', {'class':'LC20lb'}):
    print(_.text.strip(), _.find_parents('a')[0]['href'])

[Error] https://www.google.com/search
파이썬 - 위키백과, 우리 모두의 백과사전 https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC
파이썬 자습서 — Python 3.8.4 문서 https://docs.python.org/ko/3/tutorial/index.html
Python - 나무위키 https://namu.wiki/w/Python
1 파이썬 시작하기 - 왕초보를 위한 Python - WikiDocs https://wikidocs.net/43
Python 코딩의 기초 - 예제로 배우는 파이썬 프로그래밍 http://pythonstudy.xyz/python/article/6-Python-%EC%BD%94%EB%94%A9%EC%9D%98-%EA%B8%B0%EC%B4%88
파이썬 코딩 도장: 1.3 파이썬 https://dojang.io/mod/page/view.php?id=2153
파이썬의 인기는 언제까지 갈까? - Byline Network https://byline.network/2020/03/17-90/
파이썬 무료 강의 (기본편) - 6시간 뒤면 나도 개발자 - 인프런 https://www.inflearn.com/course/%EB%82%98%EB%8F%84%EC%BD%94%EB%94%A9-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EA%B8%B0%EB%B3%B8


### Selector

In [20]:
for _ in dom.select('div.r > a'):
    print(_['href'], _.text.strip())

https://ko.wikipedia.org/wiki/%ED%8C%8C%EC%9D%B4%EC%8D%AC 파이썬 - 위키백과, 우리 모두의 백과사전ko.wikipedia.org › wiki › 파이썬
https://docs.python.org/ko/3/tutorial/index.html 파이썬 자습서 — Python 3.8.4 문서docs.python.org › tutorial
https://namu.wiki/w/Python Python - 나무위키namu.wiki › Python
https://wikidocs.net/43 1 파이썬 시작하기 - 왕초보를 위한 Python - WikiDocswikidocs.net › ...
http://pythonstudy.xyz/python/article/6-Python-%EC%BD%94%EB%94%A9%EC%9D%98-%EA%B8%B0%EC%B4%88 Python 코딩의 기초 - 예제로 배우는 파이썬 프로그래밍pythonstudy.xyz › python › article › 6-Python-코딩의-...
https://dojang.io/mod/page/view.php?id=2153 파이썬 코딩 도장: 1.3 파이썬dojang.io › 강좌 › 프로그래밍 › 파이썬 코딩 도장
https://byline.network/2020/03/17-90/ 파이썬의 인기는 언제까지 갈까? - Byline Networkbyline.network › 2020/03
https://www.inflearn.com/course/%EB%82%98%EB%8F%84%EC%BD%94%EB%94%A9-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EA%B8%B0%EB%B3%B8 파이썬 무료 강의 (기본편) - 6시간 뒤면 나도 개발자 - 인프런www.inflearn.com › course › 나도코딩-파이썬-기본


In [21]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
params['query'] = '파 이 썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')

for _ in dom.select('.type01 dt > a'):
    print(_['href'], _.text.strip())

[Error] https://search.naver.com/search.naver
https://www.python.org/ Welcome to Python.org
https://papago.naver.net/website?source=en&target=ko&url=https://www.python.org/ 
https://blog.naver.com/icd900/221318575090 국산 중형차 추천 흰 파 썬
https://blog.naver.com/sanaiauction/221407173464 2015년형 그랜저HG 240모던 흰파썬 소개합니다!
https://blog.naver.com/xmflxhs8568?Redirect=Log&logNo=221532773930 레인보우 썬팅 전면,측.후면 12% 파.썬 30%
https://blog.naver.com/choyooney?Redirect=Log&logNo=221408338615 늦잠,티타임,GA과제,파에썬
https://blog.naver.com/sanaiauction?Redirect=Log&logNo=221407173464 2015년형 그랜저HG 240모던 흰파썬 소개합니다!
https://blog.naver.com/va85hgnmvcl?Redirect=Log&logNo=221687347280 [미얀마 만달레이 호텔] 썬 파 트래블 & 투어 인접한 더...
https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=357557482&qb=7YyMIOydtCDsjaw=&enc=utf8§ion=kin&rank=1&search_sort=0&spq=0 파이썬 if 문에서 and로 연결하는 방법
https://kin.naver.com/qna/detail.nhn?d1id=1&dirId=10402&docId=342332721&qb=7YyMIOydtCDsjaw=&enc=utf8§ion=kin&rank=2&search_sort=0&spq=0 파이썬 질문 ㅜㅜㅜ
http

In [22]:
url = 'http://example.webscraping.com/places/default/index'
resp = download(url)
dom = BeautifulSoup(resp.content, 'html.parser')

In [23]:
for _ in dom.select('a'):
    print(_['href'])

#
/places/default/user/register?_next=/places/default/index
/places/default/user/login?_next=/places/default/index
/places/default/index
/places/default/search
/places/default/view/Afghanistan-1
/places/default/view/Aland-Islands-2
/places/default/view/Albania-3
/places/default/view/Algeria-4
/places/default/view/American-Samoa-5
/places/default/view/Andorra-6
/places/default/view/Angola-7
/places/default/view/Anguilla-8
/places/default/view/Antarctica-9
/places/default/view/Antigua-and-Barbuda-10
/places/default/index/1


### Crawler

In [24]:
urls = list()
seen = list()
urls.append(url)

while urls: # Queue
    seed = urls.pop(0) # starting url
    seen.append(seed) # => 재방문 회피
    dom = BeautifulSoup(download(seed).text, 'html.parser') # HTTP
#     for _ in dom.select('a'): # extract hyperlinks
#         if _.has_attr('href'): # 나중에
#             if _['href'].startswith('/'): # filter1
#                 newUrls = urljoin(seed, _['href']) # Normalization
#                 # query부분 (GET방식에서 ? 이후에 나오는 파라미터 생략)
                
#                 if newUrls not in seen and newUrls not in urls:
#                     urls.append(urljoin(seed, _['href']))
    for _ in [_['href'] for _ in dom.select('a') if _.has_attr('href') and _['href'].startswith('/')]:
            newUrls = urljoin(seed, urlparse(_)[2])
            if newUrls not in seen and newUrls not in urls:
                urls.append(newUrls)
    print(len(urls), len(seen))
    if len(seen) > 10: break

14 1
13 2
12 3
11 4
18 5
20 6
26 7
34 8
35 9
37 10
41 11


### 정규식

In [25]:
import re

data = '''
park 800905-1049118
kim 700905-1059119
'''

pattern = re.compile('(\d{6})[-]\d{7}')
pattern.search(data)
print(pattern.sub('\g<1>-*******', data))


park 800905-*******
kim 700905-*******



In [26]:
re.match(r'Crow|Servo', 'CrowHello')
re.search(r'^Crow|^Servo', 'Servo')
re.search(r'Hello$', 'asdfdasHello')

<re.Match object; span=(7, 12), match='Hello'>

In [27]:
pattern.search(data)

<re.Match object; span=(6, 20), match='800905-1049118'>

In [28]:
re.search(r'(ABC)+', 'ABCABCA A').group()

'ABCABC'

In [29]:
re.search(r'\bclass\b', 'no class at all')

<re.Match object; span=(3, 8), match='class'>

In [30]:
re.search(r'\Bclass\b', 'one subclass at all')

<re.Match object; span=(7, 12), match='class'>

In [31]:
re.search(r'\Bclass\B', 'one subclasses at all')

<re.Match object; span=(7, 12), match='class'>

In [32]:
result = re.search(r'(\w+) (\w+)', 'Issac Newton, physicist')
result.group()
result.group(0), result.group(1), result.group(2)

('Issac Newton', 'Issac', 'Newton')

In [33]:
re.sub(r'(\w+) (\w+)', r'\g<2> \g<1>', 'Issac Newton, physicist')

'Newton Issac, physicist'

In [34]:
# 010-0000-0000 => 핸드폰 번호

data = '010-0000-0000'
data = '010 0000 0000'
data = '010.0000.0000'
data = '01000000000'
data = '0234081234'

re.search(r'(\d{1,3})[- .]?(\d{1,4})[- .]?(\d{1,4})',data)

<re.Match object; span=(0, 10), match='0234081234'>

In [35]:
data = '+82 10-0001-0002'
# data = '+82 010-0001-0002'
# data = '+82 2-0001-0002'
# data = '82 2-0001-0002'

re.search(r'([+]?\d{2}[ ](\d{1,3})[- .]?(\d{4})[- .]?(\d{4}))', data)

<re.Match object; span=(0, 16), match='+82 10-0001-0002'>

#### example

In [36]:
resp = download('http://pythonscraping.com/pages/page3.html')

In [37]:
re.findall(r'([$]\d+[.]\d+)', resp.text)

['$5.00', '$15.00', '$0.50', '$1.50']

In [38]:
re.findall(r'src="(.+?)"', resp.text)

['../img/gifts/logo.jpg',
 '../img/gifts/img1.jpg',
 '../img/gifts/img2.jpg',
 '../img/gifts/img3.jpg',
 '../img/gifts/img4.jpg',
 '../img/gifts/img6.jpg']

In [39]:
for _ in re.findall(r'src="(.+img\d[.]jpg)"', resp.text):
    print(urljoin(resp.request.url, _))

http://pythonscraping.com/img/gifts/img1.jpg
http://pythonscraping.com/img/gifts/img2.jpg
http://pythonscraping.com/img/gifts/img3.jpg
http://pythonscraping.com/img/gifts/img4.jpg
http://pythonscraping.com/img/gifts/img6.jpg


In [40]:
re.findall(r'<span class="[A-z]+">[A-z !]+</span>', resp.text)

['<span class="excitingNote">Keep your friends guessing!</span>']

In [41]:
dom = BeautifulSoup(resp.text, 'html.parser')
[_.strip() for _ in dom.find_all(text=re.compile(r'[.]\d+$'))]

['$15.00', '$10,000.52', '$10,005.00', '$0.50', '$1.50']

In [42]:
# 이메일
# 아이디@이메일
# 회사명.com|net|org|kr|co.kr|or.kr|go.kr|ac.kr
# 아이디 규칙 : 영어로 시작, 특수문자X, -. _. 숫자

In [43]:
email = '''
    sunnight9507@naver.com
    test1@test.com
    test2@test.co.kr
    test3@email.test.com
    test4@email.test.co.kr
    한글@test.com
    0test@test.com
'''

re.findall(r'\b[A-Za-z][A-Za-z0-9-_.]+@(?:[.]?[A-Za-z0-9-_]+)+',email)

['sunnight9507@naver.com',
 'test1@test.com',
 'test2@test.co.kr',
 'test3@email.test.com',
 'test4@email.test.co.kr']

In [44]:
# 도메인
# http://도메인
# https://도메인 
# ftp://도메인

In [45]:
domain = '''
    http://www.domain.com
    http://www.domain.co.kr
    https://www.domain.com
    https://www.domain.co.kr
    https://www.domain.co.kr/
    https://www.domain.co.kr/service
    https://www.domain.co.kr/service/
    https://domain.co.kr/service?key=value
    https://www.domain.co.kr/service?key=value
    https://www.domain.co.kr/service?key=value/
    https://www.domain.co.kr/service/1/2?key=value/
    https://www.domain.co.kr/service/1/2?key=value/?key=value
    www.domain.com?key=value
    http://old.mail.domain.com?key=value
    http://domain.com#key=value
    http://www.domain.com?key=value
'''
# Query String (?이후부분) 빼고, URI(L)을 추출
re.findall(r'http[s]?://[w]{0,3}.[A-Za-z./0-9]+', domain)

['http://www.domain.com',
 'http://www.domain.co.kr',
 'https://www.domain.com',
 'https://www.domain.co.kr',
 'https://www.domain.co.kr/',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service/',
 'https://domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service/1/2',
 'https://www.domain.co.kr/service/1/2',
 'http://old.mail.domain.com',
 'http://domain.com',
 'http://www.domain.com']

In [46]:
re.findall(r'https?://\w+(?:[./]\w+)+',domain)

['http://www.domain.com',
 'http://www.domain.co.kr',
 'https://www.domain.com',
 'https://www.domain.co.kr',
 'https://www.domain.co.kr',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service',
 'https://www.domain.co.kr/service/1/2',
 'https://www.domain.co.kr/service/1/2',
 'http://old.mail.domain.com',
 'http://domain.com',
 'http://www.domain.com']

### Google Crawler

In [47]:
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')

[Error] https://www.google.com/search


In [65]:
re.findall(r'\b파이썬 [가-힣 ]+\b',resp.text)

['파이썬 입문 강좌 무료',
 '파이썬 다운',
 '파이썬 강좌',
 '파이썬 설치',
 '파이썬 사이트',
 '파이썬 독학',
 '파이썬 활용',
 '파이썬 프로그램',
 '파이썬 소프트웨어 재단',
 '파이썬 자습서',
 '파이썬 인터프리터 사용하기',
 '파이썬 시작하기',
 '파이썬 코딩 무료 강의',
 '파이썬 기초 강좌',
 '파이썬 프로그래밍',
 '파이썬 코딩 도장',
 '파이썬 코딩 도장',
 '파이썬 코딩 도장',
 '파이썬 무료 강의',
 '파이썬 다운',
 '파이썬 강좌',
 '파이썬 설치',
 '파이썬 사이트',
 '파이썬 독학',
 '파이썬 활용',
 '파이썬 프로그램',
 '파이썬 뜻',
 '파이썬 의미',
 '파이썬 장점',
 '파이썬 소프트웨어 재단',
 '파이썬 설치']

In [89]:
url = 'https://www.google.com/search'
params = {
    'q':'',
    'oq':'',
    'aqs':'chrome..69i57j69i59j69i65l3j69i61j69i60j69i61.1205j0j7',
    'sourceid':'chrome',
    'ie':'UTF-8'
}
params['q'] = params['oq'] = '파이썬'
headers = {
    'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'
}
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
urls = [{'url': _['href'], 'depth':1}
        for _ in dom.select('div.r > a[href]')]

[Error] https://www.google.com/search


In [90]:
len(urls)

8

In [91]:
seen = list()
# urls.append(url)

while urls: # Queue
    seed = urls.pop(0) # starting url
    seen.append(seed) # => 재방문 회피
    if seed['depth'] < 2:
        dom = BeautifulSoup(download(seed['url']).text, 'html.parser') # HTTP
        for _ in [_['href'] for _ in dom.select('a[href]') 
                  if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', _['href'])]:
                newUrls = urljoin(seed['url'], urlparse(_)[2])
                if newUrls not in [_['url'] for _ in seen] and newUrls not in [_['url'] for _ in urls]:
                    urls.append({'url' : newUrls, 'depth' : seed['depth']+1})
        print(len(urls), len(seen))

135 1
138 2
[Error] https://namu.wiki/w/Python
308 3
[Error] https://wikidocs.net/43
308 4
342 5
351 6
398 7
529 8


### Naver Crawler

In [105]:
url = 'https://search.naver.com/search.naver'
params = {
    'sm':'top_hty',
    'fbm':0,
    'ie':'utf8',
    'query':''
}
params['query'] = '파 이 썬'
resp = download(url, params, headers, 'GET')
dom = BeautifulSoup(resp.text, 'html.parser')
urls = [{'url':_['href'], 'depth':1}
       for _ in dom.select('div.blog.section._blogBase._prs_blg dt > a[href]')]

[Error] https://search.naver.com/search.naver


In [106]:
urls

[{'url': 'https://blog.naver.com/xmflxhs8568?Redirect=Log&logNo=221532773930',
  'depth': 1},
 {'url': 'https://blog.naver.com/choyooney?Redirect=Log&logNo=221408338615',
  'depth': 1},
 {'url': 'https://blog.naver.com/sanaiauction?Redirect=Log&logNo=221407173464',
  'depth': 1},
 {'url': 'https://blog.naver.com/va85hgnmvcl?Redirect=Log&logNo=221687347280',
  'depth': 1}]

In [111]:
seen = list()
# urls.append(url)

while urls: # Queue
    seed = urls.pop(-1) # starting url
    seen.append(seed) # => 재방문 회피
#     if seed['depth'] < 2: # 도메인 제한 => Focused Crawling
    dom = BeautifulSoup(download(seed['url']).text, 'html.parser') # HTTP
    for _ in [_['href'] if _.has_attr('href') else _['src']
              for _ in dom.select('a[href], #mainFrame') 
              if re.match(r'(?:https?:/)?/\w+(?:[./]\w+)+', 
                          _['href'] if _.has_attr('href') else _['src'])]:
            newUrls = urljoin(seed['url'], _)
            if newUrls not in [_['url'] for _ in seen] and newUrls not in [_['url'] for _ in urls]:
                urls.append({'url' : newUrls, 'depth' : seed['depth']+1})
    print(len(urls), len(seen))
    break

1 1


In [112]:
urls

[{'url': 'https://blog.naver.com/PostView.nhn?blogId=xmflxhs8568&logNo=221532773930&from=search&redirect=Log&widgetTypeCall=true&directAccess=false',
  'depth': 2}]

In [113]:
seen

[{'url': 'https://blog.naver.com/xmflxhs8568?Redirect=Log&logNo=221532773930',
  'depth': 1}]

In [114]:
newUrls

'https://blog.naver.com/PostView.nhn?blogId=xmflxhs8568&logNo=221532773930&from=search&redirect=Log&widgetTypeCall=true&directAccess=false'